In [29]:
# Import libraries 
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone 
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv
load_dotenv()
import os

In [30]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV')

In [33]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [34]:
extracted_data = load_pdf('/Users/mhlaghari/Documents/Documents/MyProjects/end-to-end-medical-chatbot-using-llama2_v3/data/')

In [35]:
# Create text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [36]:
text_chunks = text_split(extracted_data)
print('length of my text chucks:', len(text_chunks))

length of my text chucks: 6983


In [37]:
# Download embedding model 
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [38]:
embeddings = download_hugging_face_embeddings()

In [39]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [40]:
query_result = embeddings.embed_query('Hello World')
print("Length", len(query_result))

Length 384


In [41]:
# Initializing Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
             environment=PINECONE_API_ENV)

index_name='medical-chatbot'

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], 
                                embeddings, 
                                index_name=index_name)

In [42]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = 'What are Allergies'

docs= docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.KEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction —An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny nose.\nConjunctiva —The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.', metadata={}), Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.KEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction —An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny nose.\nConjunctiva —The mucous membrane that covers\nthe white part of the eyes

In [43]:
prompt_template = '''
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else
Helpful answer:
'''

In [44]:
PROMPT = PromptTemplate(template=prompt_template,
                       input_variables=['context', 'question'])
chain_type_kwargs={'prompt': PROMPT}

In [45]:
llm = CTransformers(
    model='/Users/mhlaghari/Documents/Documents/MyProjects/end-to-end-medical-chatbot-using-llama2_v3/model/llama-2-7b-chat.ggmlv3.q4_0.bin',
    model_type='llama',
config={'max_new_tokens': 512,
        'temperature': 0.8})

In [46]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [47]:
while True:
    user_input=input(f'Input Prompt:')
    result=qa({'query': user_input})
    print("Response:", result['result'])

KeyboardInterrupt: 